In [ ]:
import pandas as pd
import os
import requests
import base64
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv

In [15]:
load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [16]:
# Load dataset to pandas Dataframe
df = pd.read_csv("hf://datasets/ailsntua/Chordonomicon/chordonomicon_v2.csv", low_memory=False)

In [3]:
# Drop rows that do not contain spotify song id and chords
mask = df["chords"].notna() & df["spotify_song_id"].notna() & df["genres"].notna()

clean_df = df[mask].copy()

In [5]:
print(clean_df.loc[1, "chords"])

<intro_1> E D A/Cs E D A/Cs <verse_1> E D A/Cs E D A/Cs E D A/Cs E D A C <chorus_1> E G D A E G D A E G D A C D E D A/Cs <verse_2> E D A/Cs E D A/Cs E D A/Cs E D A C <chorus_2> E G D A E G D A E G D A C D <bridge_1> E C G D E C G D E C G D C D E G E G D A E G D A E G D A C D <chorus_3> E G D A E G D A E G D A C D <bridge_2> E C G D E C G D E C G D C D E


In [6]:
type(clean_df.loc[1, "chords"])

str

In [ ]:
# Get Spotify API token
token_res = requests.post(
    url="https://accounts.spotify.com/api/token",
    headers={
        "Content-Type": "application/x-www-form-urlencoded",
    },
    data={
        "grant_type": "client_credentials",
        "client_id": SPOTIFY_CLIENT_ID,
        "client_secret": SPOTIFY_CLIENT_SECRET,
    }
)

In [21]:
token_res.json()

{'access_token': 'BQDjMbVLhOs7tIGvOsetwmapFiCOzxr6xwQ9T7x0D-vAk1NFNovNWjwPFG7ZNll6jtQ5okLT7VOBCVNzJc66TxAcoKUXDDKmReO-IHZW2zqEqcSo0aJzCq9vCfmeSeTlpW-OClkVBO0',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [ ]:
# test requesting full track data from spotify api
# request_full_track_data = requests.get(
#     url=f"https://api.spotify.com/v1/tracks/{clean_df.loc[626507, 'spotify_song_id']}",
#     headers={
#         "Authorization": f"Bearer {token_res.json()['access_token']}"
#     }
# )


# Test search for summer by keshi
mask = df["spotify_song_id"] == "0LqCvXVNvcnkq78sCQsoO0"

print(clean_df.loc[mask, "spotify_song_id"])

request_full_track_data = requests.get(
    url=f"https://api.spotify.com/v1/tracks/{clean_df.loc[mask, 'spotify_song_id'].values[0]}",
    headers={
        "Authorization": f"Bearer {token_res.json()['access_token']}"
    }
)

626506    0LqCvXVNvcnkq78sCQsoO0
Name: spotify_song_id, dtype: object
